# Ions in a Flame

In this example we simulate a freely-propagating, adiabatic, 1-D flame and
* Calculate ions concentration in a flame

The figure below illustrates the setup, in a flame-fixed co-ordinate system. The reactants enter with density $\rho_{u}$, temperature $T_{u}$ and speed $S_{u}$. The products exit the flame at speed $S_{b}$, density $\rho_{b}$ and temperature $T_{b}$.

<img src="images/flameSpeed.png" alt="Freely Propagating Flame" style="width: 300px;"/>

In [2]:
from __future__ import print_function
from __future__ import division

import cantera as ct
import numpy as np

print("Running Cantera Version: " + str(ct.__version__))

Running Cantera Version: 2.4.0a1


### Define the reactant conditions, gas mixture and kinetic mechanism associated with the gas

In [3]:
#Inlet Temperature in Kelvin and Inlet Pressure in Pascals
#In this case we are setting the inlet T and P to room temperature conditions
To = 300
Po = 101325

#Define the gas-mixutre and kinetics
#In this case, we are choosing a GRI3.0 gas
gas = ct.Solution('ch4_plasma.xml')

# Create a stoichiometric CH4/Air premixed mixture 
gas.set_equivalence_ratio(1.0, 'CH4', {'O2':1.0, 'N2':3.76})
gas.TP = To, Po

### Define flame simulation conditions

In [4]:
# Domain width in metres
width = 0.05

# Create the flame object
f = ct.IonFlame(gas, width=width)

# Define tolerances for the solver
f.set_refine_criteria(ratio=3, slope=0.06, curve=0.12)

# Define logging level
loglevel = 1

### Solve

In [5]:
# phase one
f.ionSolve(loglevel=loglevel, auto=True)


************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     2.136e-05       4.89
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001825      5.321
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      4.33e-05      5.965
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.00111      3.415
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point grid(s).

..............................................................................
grid refinement disabled.

******************** Solving with grid refinement enabled ********************

..............................................................................
Attempt Newton solution of steady-state probl

In [ ]:
# turn on plasma
f.plasma_enabled = True
f.flame.set_transversElecField(2.5e4)
f.flame.set_plasmaSourceMultiplier(0.001)

In [ ]:
# phase two
f.ionSolve(loglevel=loglevel, stage=2, enable_energy=False)
f.ionSolve(loglevel=loglevel, stage=2, enable_energy=True)

In [ ]:
f.flame.set_plasmaSourceMultiplier(0.01)
f.ionSolve(loglevel=loglevel, stage=2, enable_energy=True)

In [ ]:
f.flame.set_plasmaSourceMultiplier(0.1)
f.ionSolve(loglevel=loglevel, stage=2, enable_energy=True)

In [ ]:
f.flame.set_plasmaSourceMultiplier(1.0)
f.ionSolve(loglevel=loglevel, stage=2, enable_energy=True)

In [ ]:
f.ionSolve(loglevel=loglevel, stage=3, enable_energy=True)

### Plot figures

Check and see if all has gone well. Plot temperature and species fractions to see

In [ ]:
# Import plotting modules and define plotting preference
import matplotlib.pylab as plt
%matplotlib notebook

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.figsize'] = (8,6)

# Get the best of both ggplot and seaborn
plt.style.use('ggplot')
plt.style.use('seaborn-deep')

plt.rcParams['figure.autolayout'] = True

### Import Modules

#### Temperature Plot

In [ ]:
plt.figure()

plt.plot(flame.grid*100, flame.T, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Temperature (K)');

In [ ]:
plt.figure()
plt.plot(flame.grid*100, flame.phi, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Electric Potential (V)');

In [ ]:
plt.figure()
plt.plot(flame.grid*100, flame.E, '-o')
plt.xlabel('Distance (cm)')
plt.ylabel('Electric Field (V/m)');

#### Charged species' plot

In [ ]:
"""
# To plot species, we first have to identify the index of the species in the array
# For this, cut & paste the following lines and run in a new cell to get the index
for i, specie in enumerate(gas.species()):
    print(str(i) + '. ' + str(specie))
"""

# Extract concentration data
X_HCOp = flame.X[53]
X_H3Op = flame.X[54]
X_E = flame.X[55]

plt.figure()

plt.plot(flame.grid*100, X_HCOp*1000, '-o', label=r'$HCO^{+}(X{1000})$')
plt.plot(flame.grid*100, X_H3Op, '-s', label=r'$H3O^{+}$')
plt.plot(flame.grid*100, X_E, '-<', label=r'$E$')

plt.legend(loc=2)
plt.xlim([1.5,2.25])
plt.xlabel('Distance (cm)')
plt.ylabel('MoleFractions');